In [2]:
import nltk
nltk.download('punkt')  # Required for tokenizing sentences for BLEU scoring


[nltk_data] Downloading package punkt to /home/admincit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
from transformers import Blip2ForConditionalGeneration, AutoProcessor
import torch
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Load the model and processor from your saved directory
processor = AutoProcessor.from_pretrained("./BLIP_finetuned_dora_r16/checkpoint_26")
model = Blip2ForConditionalGeneration.from_pretrained("./BLIP_finetuned_dora_r16/checkpoint_26")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()


/home/admincit/DoRA-LoRA/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:02<00:00,  3.39it/s]


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [2]:
from datasets import load_dataset 

dataset = load_dataset("Pranavkpba2000/skin_cancer_small_dataset", split="test")

0

In [13]:
index=2000
# prepare image for the model
inputs = processor(images=dataset[index]['image'], return_tensors="pt").to("cuda", torch.float16)
pixel_values = inputs.pixel_values

generated_ids = model.generate(pixel_values=pixel_values, max_length=25)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(generated_caption)

5
The detected disease is Squamous Cell Carcinoma (SCC), a type of skin cancer that often appears as


In [7]:
def evaluate_model(model, processor, dataset):
    bleu_scores = []
    rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    # Initialize ROUGE scorer
    rouge_scorer_obj = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    for item in dataset:
        # Process image and prompt target
        inputs = processor(images=item["image"], return_tensors="pt").to(device)
        target_prompt = item["prompt"]  # Replace with actual prompt in dataset

        # Generate a caption with the model
        with torch.no_grad():
            generated_ids = model.generate(**inputs)
            generated_caption = processor.decode(generated_ids[0], skip_special_tokens=True)

        # Compute BLEU score
        reference = target_prompt.split()  # Convert to token list for BLEU
        candidate = generated_caption.split()
        bleu_score = sentence_bleu([reference], candidate)
        bleu_scores.append(bleu_score)

        # Compute ROUGE scores
        rouge_result = rouge_scorer_obj.score(target_prompt, generated_caption)
        rouge_scores["rouge1"].append(rouge_result["rouge1"].fmeasure)
        rouge_scores["rouge2"].append(rouge_result["rouge2"].fmeasure)
        rouge_scores["rougeL"].append(rouge_result["rougeL"].fmeasure)

    # Calculate average scores
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge1 = sum(rouge_scores["rouge1"]) / len(rouge_scores["rouge1"])
    avg_rouge2 = sum(rouge_scores["rouge2"]) / len(rouge_scores["rouge2"])
    avg_rougeL = sum(rouge_scores["rougeL"]) / len(rouge_scores["rougeL"])

    print("Evaluation Metrics:")
    print(f"BLEU Score: {avg_bleu:.4f}")
    print(f"ROUGE-1 Score: {avg_rouge1:.4f}")
    print(f"ROUGE-2 Score: {avg_rouge2:.4f}")
    print(f"ROUGE-L Score: {avg_rougeL:.4f}")

# Example usage
evaluate_model(model, processor, dataset)


KeyError: 'prompt'

In [1]:
from datasets import load_dataset 

dataset = load_dataset("Pranavkpba2000/skin_cancer_small_dataset", split="test")

# Assuming dataset is a list of dictionaries like:
# dataset = [{"image": image_data, "label": 0}, {"image": image_data, "label": 1}, ...]

from collections import defaultdict

def extract_images_per_class(dataset, num_images_per_class=1):
    """
    Extracts a specified number of images from each class and creates a new dataset.

    Args:
    - dataset: Original dataset containing images and labels.
    - num_images_per_class: The number of images to extract from each class.
    
    Returns:
    - new_dataset: A new dataset with the extracted images and labels.
    """
    # Dictionary to store images by class (labels 0-7)
    class_to_images = defaultdict(list)

    # Group images by their labels
    for item in dataset:
        class_to_images[item["label"]].append(item)

    # Create a new dataset with only 4 images per class (0 to 7)
    new_dataset = []
    for label in range(8):  # There are 8 labels (0 to 7)
        # Get the images for the current label, and ensure we take at most 4
        images_of_class = class_to_images[label][:num_images_per_class]

        # Add these images to the new dataset
        new_dataset.extend(images_of_class)

    return new_dataset

# Example usage:
new_dataset = extract_images_per_class(dataset)


/home/admincit/DoRA-LoRA/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from nltk.translate.bleu_score import sentence_bleu
from transformers import Blip2ForConditionalGeneration, AutoProcessor
from datasets import load_dataset
from rouge_score import rouge_scorer
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
import torch

# Load model and processor
processor = AutoProcessor.from_pretrained("./BLIP_finetuned_dora_r16/checkpoint_30")
model = Blip2ForConditionalGeneration.from_pretrained("./BLIP_finetuned_dora_r16/checkpoint_30")

# Load validation dataset
validation_dataset = new_dataset

# Define label to prompt mapping for evaluation
label_to_prompt = {
    0: "The detected disease is Actinic Keratosis (AK), a precancerous condition characterized by scaly, crusty patches of skin. It can potentially develop into skin cancer if not treated.",
    1: "The detected disease is Basal Cell Carcinoma (BCC), a common form of skin cancer that typically appears as a pearly or waxy bump. It is slow-growing and usually doesn't spread.",
    2: "The detected disease is Benign Keratosis (BKL), a non-cancerous skin growth that often appears as a wart or mole. These are usually harmless but may need monitoring.",
    3: "The detected disease is Melanoma (MEL), a serious and aggressive form of skin cancer that often presents as a mole with irregular edges or different colors. It requires immediate attention.",
    4: "The detected disease is Nevus (NV), commonly known as a mole, which can vary in color and size. Most are harmless, but changes in size, shape, or color may require evaluation.",
    5: "The detected disease is Squamous Cell Carcinoma (SCC), a type of skin cancer that often appears as a firm, red nodule or scaly patch. It can spread if not treated early.",
    6: "The detected disease is Dermatofibroma (DF), a benign growth on the skin that is typically brown or tan. These are harmless and generally do not require treatment.",
    7: "The detected disease is Vascular Lesions (VASC), abnormal blood vessel growths that can appear as red or purple spots on the skin. They are generally harmless but may require treatment for cosmetic reasons."
}

# Initialize evaluators
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
cider_evaluator = Cider()
meteor_evaluator = Meteor()

# Function to evaluate BLEU, ROUGE, CIDEr, and METEOR scores on validation set
def evaluate_metrics(model, processor, validation_dataset):
    model.eval()
    total_bleu_score = 0
    total_rouge_score = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    total_cider_score = 0
    total_meteor_score = 0
    count = 0

    for item in validation_dataset:
        image = item["image"]
        label = item["label"]

        # Generate caption
        inputs = processor(images=image, return_tensors="pt").to(model.device)
        generated_ids = model.generate(**inputs,max_length=80)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Reference prompt
        reference_text = label_to_prompt[label]

        # Calculate BLEU score
        reference_tokens = reference_text.split()
        generated_tokens = generated_text.split()
        bleu_score = sentence_bleu([reference_tokens], generated_tokens)
        total_bleu_score += bleu_score

        # Calculate ROUGE scores
        rouge_scores = rouge_scorer_instance.score(reference_text, generated_text)
        for key in total_rouge_score:
            total_rouge_score[key] += rouge_scores[key].fmeasure

        # Calculate CIDEr score (CIDEr expects a list of dicts)
        cider_score, _ = cider_evaluator.compute_score(
            {0: [reference_text]}, {0: [generated_text]}
        )
        total_cider_score += cider_score

        # Calculate METEOR score
        meteor_score, _ = meteor_evaluator.compute_score(
            {0: [reference_text]}, {0: [generated_text]}
        )
        total_meteor_score += meteor_score

        count += 1

        print(f"Image Label: {label}")
        print(f"Reference Text: {reference_text}")
        print(f"Generated Text: {generated_text}")
        print(f"BLEU Score: {bleu_score}")
        print(f"ROUGE Scores: {rouge_scores}")
        print(f"CIDEr Score: {cider_score}")
        print(f"METEOR Score: {meteor_score}\n")

    # Calculate average scores
    average_bleu_score = total_bleu_score / count
    average_rouge_score = {k: v / count for k, v in total_rouge_score.items()}
    average_cider_score = total_cider_score / count
    average_meteor_score = total_meteor_score / count

    print(f"Average BLEU Score: {average_bleu_score:.4f}")
    print(f"Average ROUGE Scores: {average_rouge_score}")
    print(f"Average CIDEr Score: {average_cider_score:.4f}")
    print(f"Average METEOR Score: {average_meteor_score:.4f}")

    return {
        "average_bleu_score": average_bleu_score,
        "average_rouge_score": average_rouge_score,
        "average_cider_score": average_cider_score,
        "average_meteor_score": average_meteor_score
    }

# Example usage
average_metrics = evaluate_metrics(model, processor, validation_dataset)


Loading checkpoint shards: 100%|██████████| 8/8 [00:02<00:00,  3.51it/s]
Expanding inputs for image tokens in BLIP-2 should be done in processing. Please follow instruction here (https://gist.github.com/zucchini-nlp/e9f20b054fa322f84ac9311d9ab67042) to update your BLIP-2 model. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


Image Label: 0
Reference Text: The detected disease is Actinic Keratosis (AK), a precancerous condition characterized by scaly, crusty patches of skin. It can potentially develop into skin cancer if not treated.
Generated Text: The detected disease is Benign Keratosis (BKL), a non-cancerous skin growth that often appears as a wart or mole. These are usually harmless but may need monitoring.
BLEU Score: 0.09992869870690119
ROUGE Scores: {'rouge1': Score(precision=0.2857142857142857, recall=0.2962962962962963, fmeasure=0.2909090909090909), 'rouge2': Score(precision=0.1111111111111111, recall=0.11538461538461539, fmeasure=0.11320754716981132), 'rougeL': Score(precision=0.25, recall=0.25925925925925924, fmeasure=0.2545454545454545)}
CIDEr Score: 0.0
METEOR Score: 0.16654299046104756

Image Label: 1
Reference Text: The detected disease is Basal Cell Carcinoma (BCC), a common form of skin cancer that typically appears as a pearly or waxy bump. It is slow-growing and usually doesn't spread.
G

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 74.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [9]:
import gradio as gr
import torch
from PIL import Image, ImageDraw, ImageFont

# Function to process and generate caption
def generate_caption(index):
    # Load the image from the dataset (ensure compatibility with PIL.Image)
    image = dataset[index]['image']
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)

    # Prepare image for the model
    inputs = processor(images=image, return_tensors="pt").to("cuda", torch.float16)
    pixel_values = inputs.pixel_values

    # Generate caption
    generated_ids = model.generate(pixel_values=pixel_values, max_length=25)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Draw the caption on the image
    img = image.convert("RGB")  # Ensure RGB format
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()

    # Wrap text to fit within the image width
    max_width = img.width - 20  # Allow for padding
    words = generated_caption.split()
    lines = []
    line = ""

    for word in words:
        test_line = f"{line} {word}" if line else word
        test_width = draw.textbbox((0, 0), test_line, font=font)[2] - draw.textbbox((0, 0), test_line, font=font)[0]
        if test_width <= max_width:
            line = test_line
        else:
            lines.append(line)
            line = word
    if line:
        lines.append(line)

    # Calculate text height and position
    line_height = draw.textbbox((0, 0), "Test", font=font)[3] - draw.textbbox((0, 0), "Test", font=font)[1]
    text_height = line_height * len(lines)
    text_position = (10, img.height - text_height - 10)  # Bottom padding

    # Draw each line of text
    for i, line in enumerate(lines):
        draw.text((10, text_position[1] + i * line_height), line, fill="white", font=font)

    return img

# Define the Gradio interface
iface = gr.Interface(
    fn=generate_caption,
    inputs=gr.Number(label="Image Index"),
    outputs=gr.Image(label="Image with Caption"),
    title="Image Caption Generator",
    description="Enter an index to view an image with its generated caption."
)

# Launch the Gradio app
iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://42db03d7b500f2c76f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
